In [1]:
import datetime
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
import ribasim.nodes

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.assign_authorities import AssignAuthorities
from peilbeheerst_model.controle_output import Control
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

from ribasim import Node
from ribasim.nodes import pump, level_boundary, tabulated_rating_curve
from shapely import Point

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [2]:
waterschap = "AmstelGooienVecht"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

ws_grenzen_path = "../../../../../Data_overig/Waterschapsgrenzen/waterschap.gpkg"
RWS_grenzen_path = "../../../../../Data_overig/Waterschapsgrenzen/Rijkswaterstaat.gpkg"

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = -0.42  # default LevelBoundary level

## Process the feedback form

In [3]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_0"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed, use_validation=True
)
processor.run()

Swapped edge direction between Node A: 70 and Node B: 451
Swapped edge direction between Node A: 451 and Node B: 20
Swapped edge direction between Node A: 225 and Node B: 714
Swapped edge direction between Node A: 714 and Node B: 1001
Swapped edge direction between Node A: 222 and Node B: 572
Swapped edge direction between Node A: 572 and Node B: 88
Processed all actions
The target levels (streefpeilen) have been updated.
The function of the pumps have been updated.


#### Load model

In [4]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

# Parameterization

## Nodes

### Basin (characteristics)

In [5]:
ribasim_param.validate_basin_area(ribasim_model)

All basins are larger than 100 m²


## Model specific tweaks

In [6]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

In [7]:
#add outlet and LB near the IJ
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(125905, 486750)), [level_boundary.Static(level=[default_level])])
tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(125958,486838)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(ribasim_model.basin[225], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

#add additional pump and LB to ARK
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(129850, 480894)), [level_boundary.Static(level=[default_level])])

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(129829, 480893)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[229], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

#add outlet and LB from ARK-NZK to Loosdrechtse Plassen
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(129097, 468241)), [level_boundary.Static(level=[default_level])])
tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(129097, 468241)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(level_boundary_node, tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, ribasim_model.basin[59])




#add outlet to Gooimeer
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(146641, 479856)), [level_boundary.Static(level=[default_level])])
tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(146592, 479749)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(ribasim_model.basin[215], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)


#add additional pump and LB to ARK
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(129677,482929)), [level_boundary.Static(level=[default_level])])

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(129674, 482974)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[228], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)


#add nood overlaat to ARK
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(128903, 470553)), [level_boundary.Static(level=[default_level])])
tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(128913, 470563)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(ribasim_model.basin[69], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [8]:
ribasim_model.level_boundary.node.df.meta_node_id = ribasim_model.level_boundary.node.df.index
ribasim_model.tabulated_rating_curve.node.df.meta_node_id = ribasim_model.tabulated_rating_curve.node.df.index
ribasim_model.pump.node.df.meta_node_id = ribasim_model.pump.node.df.index



In [9]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

In [10]:
ribasim_model.basin.area.df["meta_streefpeil"] = ribasim_model.basin.area.df["meta_streefpeil"].astype(float)

In [11]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

## Implement standard profile and a storage basin

In [12]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [13]:
# remove after the feedback forms have been fixed
ribasim_model.basin.profile.df.loc[ribasim_model.basin.profile.df.index == 559, "area"] = 1
ribasim_model.basin.profile.df.loc[ribasim_model.basin.profile.df.index == 560, "area"] = 2

In [14]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()


### Basin (forcing)

In [15]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(0),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(10),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [16]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [17]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)  # clean
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

### Add Outlet

In [18]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [19]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [20]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")


In [21]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [22]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

Warning! Some pumps do not have a flow rate yet. Dummy value of 0.1234 m3/s has been taken.


In [23]:
# overwrite some sturing
# ribasim_model.outlet.static.df.max_downstream_level = np.nan
# ribasim_model.outlet.static.df.max_downstream_level += 1000
# ribasim_model.pump.static.df.loc[ribasim_model.pump.static.df.meta_func_afvoer == 1, 'max_downstream_level'] += 1000

### Manning Resistance

In [24]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [25]:
#lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 10
ribasim_model.manning_resistance.static.df.manning_n = 0.01

## Last formating of the tables

In [26]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)

assign = AssignAuthorities(ribasim_model = ribasim_model, 
                           waterschap = waterschap, 
                           ws_grenzen_path = ws_grenzen_path, 
                           RWS_grenzen_path = RWS_grenzen_path, 
                           ws_buffer=1025, 
                           RWS_buffer=1000)
ribasim_model = assign.assign_authorities()

In [27]:
# ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id==, 'meta_from_authority'] = 
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id== 908, 'meta_from_authority'] = 'HollandsNoorderkwartier' 
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id== 2912, 'meta_from_authority'] = 'Rijkswaterstaat' 



# ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id==, 'meta_to_authority'] = 
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id== 907, 'meta_to_authority'] = 'HollandsNoorderkwartier' 
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id== 972, 'meta_to_authority'] = 'HollandsNoorderkwartier' 
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id==996, 'meta_to_authority'] = 'Rijnland' 
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id==909, 'meta_to_authority'] = 'Rijkswaterstaat' 
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id==931, 'meta_to_authority'] = 'Rijkswaterstaat' 


# Set numerical settings

In [28]:
ribasim_model.use_validation = True

In [29]:
# Write model output
# ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2025, 1, 1)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

## Run Model

## Iterate over tabulated rating curves

In [30]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], 
                              print_other=False, 
                              suffix="init")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

In [31]:
controle_output = Control(work_dir=work_dir)
indicators = controle_output.run_all()

# Write model

In [32]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)

../../../../../Ribasim_networks/Waterschappen/AmstelGooienVecht/modellen/AmstelGooienVecht_parametrized/results
The model of waterboard AmstelGooienVecht has been uploaded to the goodcloud in the directory of boezemmodel!
